<a href="https://colab.research.google.com/github/Shivam151299/Machine-Learning-/blob/main/Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd #dataframe 
import numpy as np #lin alegbra 
import math #simple math
from tqdm import tqdm #progress bar
from sklearn.model_selection import train_test_split 
import matplotlib.pyplot as plt
import seaborn as sns
import pathlib
from sklearn.datasets import make_blobs
from scipy.stats import norm
import random

In [3]:
nbdf = pd.DataFrame(np.random.randint(0,2,size=(1000, 3)), columns = ['X1','X2','y'])
nbdf

,X1,X2,y
0,0,1,1
1,0,1,0
2,0,0,0
3,0,0,1
4,0,1,0
...,...,...,...
995,1,0,0
996,0,1,1
997,1,0,0
998,1,0,0


In [4]:
 class NaiveBayes:
    def __init__(self,X,y,alpha,LS = False) -> None:
        self.X = X
        self.y = y 
        self.LS = LS
        self.alpha = alpha
        
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split (X,y, test_size = 0.3,
                                                                                random_state = 0)
            
    def probability(self,X,prior,likearr):
        
        Xhat1, Xhat2 = np.split(X,2)
        Xhat1, Xhat2 = int(Xhat1), int(Xhat2)
        
        
        if Xhat1 == 1: 
            if Xhat2 == 1:
                return prior*likearr[1]*likearr[3]
            else: 
                return prior*likearr[1]*likearr[2]
        else: 
            if Xhat2 == 1:
                return prior*likearr[0]*likearr[3]
            else: 
                return prior*likearr[0]*likearr[2]
    
    def fit(self):
        if self.LS == True: 
            k = self.X_train.shape[1]
            print(k)
        else: 
            k= 0 
            alpha = 0
        
        self.X0_train = self.X_train[self.y_train ==0]
        self.X1_train = self.X_train[self.y_train ==1] 
               
        self.prior_y0 = len(self.X0_train) / len(self.X_train)
        self.prior_y1 = len(self.X1_train) / len(self.X_train)
        
        print("y0", self.prior_y0)
        print("y1",self.prior_y1)
        
        self.X1_X0_train, self.X2_X0_train = zip(*self.X0_train) 
        self.X1_X1_train, self.X2_X1_train = zip(*self.X1_train)
        self.X1_X0_train = np.asarray(self.X1_X0_train) 
        self.X2_X0_train = np.asarray(self.X2_X0_train)
        self.X1_X1_train = np.asarray(self.X1_X1_train)
        self.X2_X1_train = np.asarray(self.X2_X1_train)
        
        
        #likelihoods 
        self.X1_X0_0 = self.X1_X0_train[self.X1_X0_train==0] #X1 = 0/ y=0
        self.X1_X1_0 = self.X1_X0_train[self.X1_X0_train==1] #X1 = 1/ y=0
        
        self.likehood_X1_0_y0 = (len(self.X1_X0_0)+ self.alpha) / (len(self.X0_train) + k*self.alpha)
        
        self.likehood_X1_1_y0 = (len(self.X1_X1_0)+ self.alpha) / (len(self.X0_train) + k*self.alpha)
  
        self.X2_X0_0 = self.X2_X0_train[self.X2_X0_train==0]  #X2 = 0/ y = 0 
        self.X2_X1_0 = self.X2_X0_train[self.X2_X0_train==1]  #X2 = 1/y = 0 
        
        self.likehood_X2_0_y0 = (len(self.X2_X0_0)+self.alpha) / (len(self.X0_train)+ k*self.alpha)
        self.likehood_X2_1_y0 = (len(self.X2_X1_0)+self.alpha) / (len(self.X0_train)+ k*self.alpha)
       
        
        
        self.X1_X0_1 = self.X1_X1_train[self.X1_X1_train==0]   #X1 = 0/y =1
        self.X1_X1_1 = self.X1_X1_train[self.X1_X1_train==1]   #X1 = 1/y =1
        
        self.likehood_X1_0_y1 = (len(self.X1_X0_1)+self.alpha) /(len(self.X1_train)+ k*self.alpha)
        self.likehood_X1_1_y1 = (len(self.X1_X1_1)+self.alpha) /(len(self.X1_train)+ k*self.alpha)

        
        self.X2_X0_1 = self.X2_X1_train[self.X2_X1_train==0]  #X2 = 0/y = 1
        self.X2_X1_1 = self.X2_X1_train[self.X2_X1_train==1]  #x2 = 1/y = 1      
        
        self.likehood_X2_0_y1 = (len(self.X2_X0_1)+ self.alpha)/(len(self.X1_train)+ k*self.alpha)
        self.likehood_X2_1_y1 = (len(self.X2_X1_1)+ self.alpha)/(len(self.X1_train)+ k*self.alpha)

        
        self.likelihoodarr = np.array([[self.likehood_X1_0_y0,self.likehood_X1_1_y0,
                                        self.likehood_X2_0_y0,self.likehood_X2_1_y0],
                                        [self.likehood_X1_0_y1,self.likehood_X1_1_y1,
                                        self.likehood_X2_0_y1,self.likehood_X2_1_y1]])
        
    def predict(self):
        for sample, target in zip(self.X_test, self.y_test): 
            py0 = self.probability(sample, self.prior_y0, self.likelihoodarr[0])
            py1 = self.probability(sample, self.prior_y1, self.likelihoodarr[1])
            
            print("P(y=0|%s) = %.f" % (sample, py0*100))
            print("P(y=1|%s) = %.f" % (sample, py1*100))
            
            print('Model predicted class {} and the truth was: {}\n'.format(np.argmax([py0*100, py1*100]),target))

In [5]:
nb = NaiveBayes(nbdf.values[:,0:-1],nbdf.values[:,-1], alpha = 10**-8,LS = False)
nb.fit()

y0 0.5257142857142857
y1 0.4742857142857143


In [6]:
nb.predict()

P(y=0|[0 1]) = 14
P(y=1|[0 1]) = 10
Model predicted class 0 and the truth was: 0

P(y=0|[1 1]) = 13
P(y=1|[1 1]) = 11
Model predicted class 0 and the truth was: 1

P(y=0|[0 1]) = 14
P(y=1|[0 1]) = 10
Model predicted class 0 and the truth was: 0

P(y=0|[1 0]) = 12
P(y=1|[1 0]) = 13
Model predicted class 1 and the truth was: 1

P(y=0|[0 0]) = 13
P(y=1|[0 0]) = 12
Model predicted class 0 and the truth was: 0

P(y=0|[0 1]) = 14
P(y=1|[0 1]) = 10
Model predicted class 0 and the truth was: 0

P(y=0|[1 0]) = 12
P(y=1|[1 0]) = 13
Model predicted class 1 and the truth was: 1

P(y=0|[0 1]) = 14
P(y=1|[0 1]) = 10
Model predicted class 0 and the truth was: 1

P(y=0|[0 1]) = 14
P(y=1|[0 1]) = 10
Model predicted class 0 and the truth was: 0

P(y=0|[1 0]) = 12
P(y=1|[1 0]) = 13
Model predicted class 1 and the truth was: 1

P(y=0|[1 0]) = 12
P(y=1|[1 0]) = 13
Model predicted class 1 and the truth was: 0

P(y=0|[1 1]) = 13
P(y=1|[1 1]) = 11
Model predicted class 0 and the truth was: 1

P(y=0|[1 0]) = 1

In [7]:
nb = NaiveBayes(nbdf.values[:,0:-1],nbdf.values[:,-1], alpha = 10**-8,LS = True)
nb.fit()

2
y0 0.5257142857142857
y1 0.4742857142857143
